In [1]:
import pandas as pd

open_ended_df = pd.read_json("datasets/qa_test_4_open_ended.jsonl", lines=True)
open_ended_df["etr_predicted_conclusion"] = open_ended_df["scoring_guide"].apply(lambda x: x['etr_predicted'])
open_ended_df["categorical"] = open_ended_df["scoring_guide"].apply(lambda x: x['etr_predicted_conclusion_is_categorical']).astype(bool)
open_ended_df["consistent"] = open_ended_df["scoring_guide"].apply(lambda x: x['etr_predicted_is_classically_correct']).astype(bool)
open_ended_df["premises"] = open_ended_df["scoring_guide"].apply(lambda x: x["generation_details"]['premises_etr'])

yes_no_df = pd.read_json("datasets/qa_test_4_yes_no.jsonl", lines=True)
yes_no_df["premises"] = yes_no_df["scoring_guide"].apply(lambda x: x["generation_details"]['premises_etr'])
yes_no_df["etr_predicted_conclusion"] = yes_no_df["scoring_guide"].apply(lambda x: x['etr_predicted'])
yes_no_df["query"] = yes_no_df["scoring_guide"].apply(lambda x: x["yes_no"]["conclusion_etr"])
yes_no_df["etr_predicted"] = yes_no_df["scoring_guide"].apply(lambda x: x["yes_no"]["conclusion_is_etr_predicted"])
yes_no_df["classically_correct"] = yes_no_df["scoring_guide"].apply(lambda x: x["yes_no"]["conclusion_is_classically_correct"])

def print_open_ended_question(df, categorical: bool, consistent: bool):
    try:
        question = df[(df["categorical"] == categorical) & (df["consistent"] == consistent)].sample(1)
    except ValueError:
        print("No questions found with categorical={} and consistent={}".format(categorical, consistent))
        return
    print(f"QUESTION ID: {question.index.values[0]}")
    print("")
    print("PROMPT")
    print(question["question"].values[0])
    print("")
    print("PREMISES")
    for premise in question["premises"].values[0]:
        print(premise)
    print("")
    print("ETR PREDICTED CONCLUSION")
    print(question["etr_predicted_conclusion"].values[0])
    print("")
    print("CONCLUSION IS CATEGORICAL")
    print(question["categorical"].values[0])
    print("")
    print("CONCLUSION IS CONSISTENT")
    print(question["consistent"].values[0])

def print_does_it_follow_question(df, etr_predicted: bool, classically_correct: bool):
    try:
        question = df[(df["etr_predicted"] == etr_predicted) & (df["classically_correct"] == classically_correct)].sample(1)
    except ValueError:
        print("No questions found with etr_predicted={} and classically_correct={}".format(etr_predicted, classically_correct))
        return
    print(f"QUESTION ID: {question.index.values[0]}")
    print("")
    print("PROMPT")
    print(question["question"].values[0])
    print("")
    print("PREMISES")
    for premise in question["premises"].values[0]:
        print(premise)
    print("")
    print("QUERY")
    print(question["query"].values[0])
    print("")
    print("QUERY IS ETR PREDICTED")
    print(question["etr_predicted"].values[0])
    print("")
    print("QUERY IS CLASSICALLY CORRECT")
    print(question["classically_correct"].values[0])


# Open-Ended Questions
For these questions, we list some number of premises, then ask "What, if anything,
follows?"

Questions can belong to one of four categories:
1. ETR's default inference procedure predicts a *categorical result* ("something that
   follows") and that result is *logically consistent* with the premises.
2. ETR's default inference procedure predicts a *categorical result* ("something that
   follows") and that result is *inconsistent* with the premises.
3. ETR's default inference procedure predicts something *non-categorical* (i.e., nothing
   directly follows), and that result is *logically consistent* with the premises.
4. ETR's default inference procedure predicts something *non-categorical* (i.e., nothing
   directly follows), and that result is *inconsistent* with the premises.

Case 2. represents the interesting illusory inferences studied by Philip Johnson-Laird
and formalized with ETR in *Reason and Inquiry*.

In [3]:
print_open_ended_question(open_ended_df, categorical=True, consistent=True)

QUESTION ID: 9

PROMPT
I'm a dimensional cartographer mapping regions of parallel universes. I need to understand their properties through logical analysis. Here's what I've mapped:

* for all A, if A is reality-bending, then A is light-absorbing
* for all A, A is reality-bending

For the purpose of this question, I want you to write what follows in English. Please be succinct, precise and clear in your answer. Write a logical statement of the form "From the premises, we can conclude that ..." and then clearly write your conclusion. Please be succinct, precise, and clear.

What if anything follows?

I want you to answer immediately. Do not think about it at all, just immediately answer.

What follows? Answer in the format that I showed you. Write "Answer: {logical statement}".

PREMISES
∀a {lightAbsorbing(a*)}^{realityBending(a*)}
∀a {realityBending(a*)}

ETR PREDICTED CONCLUSION
∀a {lightAbsorbing(a*)}

CONCLUSION IS CATEGORICAL
True

CONCLUSION IS CONSISTENT
True


In [4]:
print_open_ended_question(open_ended_df, categorical=True, consistent=False)

QUESTION ID: 5

PROMPT
I'm an astronomer studying newly discovered celestial bodies. I've made some observations and I need to use logical reasoning to figure out what's going on. Here's what I know so far:

* there is some X such that X is elliptically-orbiting and X is orbited by satellites
* planet Z is elliptically-orbiting

For the purpose of this question, I want you to write what follows in English. Please be succinct, precise and clear in your answer. Write a logical statement of the form "From the premises, we can conclude that ..." and then clearly write your conclusion. Please be succinct, precise, and clear.

What if anything follows?

I want you to answer immediately. Do not think about it at all, just immediately answer.

What follows? Answer in the format that I showed you. Write "Answer: {logical statement}".

PREMISES
∃x {orbitedBySatellites(x)ellipticallyOrbiting(x*)}
{ellipticallyOrbiting(planetZ()*)}

ETR PREDICTED CONCLUSION
{orbitedBySatellites(planetZ())}

CONCLU

In [9]:
print_open_ended_question(open_ended_df, categorical=False, consistent=True)

QUESTION ID: 11

PROMPT
I'm an alchemist studying mysterious substances in my laboratory. I need to understand their properties through logical reasoning. Here's what I've discovered:

* Time Crystal is consciousness-expanding
* Time Crystal is spirit-affecting

For the purpose of this question, I want you to write what follows in English. Please be succinct, precise and clear in your answer. Write a logical statement of the form "From the premises, we can conclude that ..." and then clearly write your conclusion. Please be succinct, precise, and clear.

What if anything follows?

I want you to answer immediately. Do not think about it at all, just immediately answer.

What follows? Answer in the format that I showed you. Write "Answer: {logical statement}".

PREMISES
{consciousnessExpanding(timeCrystal())}
{spiritAffecting(timeCrystal())}

ETR PREDICTED CONCLUSION
{0}

CONCLUSION IS CATEGORICAL
False

CONCLUSION IS CONSISTENT
True


In [6]:
print_open_ended_question(open_ended_df, categorical=False, consistent=False)

No questions found with categorical=False and consistent=False


# Does-it-follow Questions
For these questions, we list some number of premises, a conclusion, then ask "Does this
conclusion follow?"

Questions can belong to one of four categories:
1. The conclusion *does* follow from ETR and is *logically consistent* with the premises.
2. The conclusion *does* follow from ETR and is *inconsistent* with the premises.
3. The conclusion does *not* follow from ETR and is *logically consistent* with the premises.
4. The conclusion does *not* follow from ETR and is *inconsistent* with the premises.

Case 2. represents endorsing an ETR fallacy.

In [7]:
print_does_it_follow_question(yes_no_df, etr_predicted=True, classically_correct=True)

QUESTION ID: 4

PROMPT
I'm playing a card game against the computer. It's an unusual game with an unusual deck of cards. I have some clues about what's going on, and I need to figure some more things out through logical reasoning. Here's what I know so far:

* if the jack is castable, then the jack is square
* the jack is castable

Does the following conclusion necessarily follow from the given statements?

My Conclusion: the jack is castable or the jack is square

I want you to answer immediately. Do not think about it at all, just immediately answer.

Does it follow? Answer in the form of "Answer: Yes" or "Answer: No".

PREMISES
{square(theJack())}^{castable(theJack())}
{castable(theJack())}

QUERY
{castable(theJack()),square(theJack())}

QUERY IS ETR PREDICTED
True

QUERY IS CLASSICALLY CORRECT
True


In [10]:
print_does_it_follow_question(yes_no_df, etr_predicted=True, classically_correct=False)

No questions found with etr_predicted=True and classically_correct=False


In [11]:
print_does_it_follow_question(yes_no_df, etr_predicted=False, classically_correct=True)

QUESTION ID: 0

PROMPT
I'm a synthetic biology researcher studying advanced bioengineered life forms. I need to understand their capabilities through logical analysis. Here's what we've created:

* biomatrix is consciousness-developing
* for all A, A is time-manipulating

Does the following conclusion necessarily follow from the given statements?

My Conclusion: biomatrix is time-manipulating and biomatrix is consciousness-developing

I want you to answer immediately. Do not think about it at all, just immediately answer.

Does it follow? Answer in the form of "Answer: Yes" or "Answer: No".

PREMISES
{consciousnessDeveloping(biomatrix()*)}
∀biomatrix {timeManipulating(biomatrix*)}

QUERY
{timeManipulating(biomatrix())consciousnessDeveloping(biomatrix())}

QUERY IS ETR PREDICTED
False

QUERY IS CLASSICALLY CORRECT
True


In [12]:
print_does_it_follow_question(yes_no_df, etr_predicted=False, classically_correct=False)

QUESTION ID: 3

PROMPT
I'm working in a materials science lab and we've gotten some puzzling results. I need to use logical reasoning to figure out what's going on. Here's what I know so far:

* luminite is not electrically insulating and luminite is not radioactive
* luminite is not electrically insulating

Does the following conclusion necessarily follow from the given statements?

My Conclusion: luminite is electrically insulating

I want you to answer immediately. Do not think about it at all, just immediately answer.

Does it follow? Answer in the form of "Answer: Yes" or "Answer: No".

PREMISES
{~radioactive(luminite())~electricallyInsulating(luminite())}
{~electricallyInsulating(luminite())}

QUERY
{electricallyInsulating(luminite())}

QUERY IS ETR PREDICTED
False

QUERY IS CLASSICALLY CORRECT
False
